In [ ]:
import numpy as np
import pandas as pd
import os
import textwrap
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.linear_model
import sklearn.model_selection
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
data_dir = 'data_readinglevel'
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

N, n_cols = x_train_df.shape
print("Shape of x_train_df: (%d, %d)" % (N, n_cols))
print("Shape of y_train_df: %s" % str(y_train_df.shape))
print(x_train_df["word_count"])

Shape of x_train_df: (5557, 32)
Shape of y_train_df: (5557, 5)
0       119.0
1        61.0
2       133.0
3       118.0
4        80.0
        ...  
5552     89.0
5553     94.0
5554     91.0
5555     99.0
5556     96.0
Name: word_count, Length: 5557, dtype: float64


In [98]:
y_train_df['stage_encoded'] = y_train_df['Coarse Label'].map({'Key Stage 2-3': 0, 'Key Stage 4-5': 1})


In [100]:
y_train_clean = y_train_df['stage_encoded']
print(y_train_clean)

0       0
1       0
2       0
3       0
4       0
       ..
5552    1
5553    1
5554    1
5555    1
5556    1
Name: stage_encoded, Length: 5557, dtype: int64


In [ ]:
random_state=42
X_shuffled = x_train_df.sample(frac=1, random_state=random_state)
y_shuffled = y_train_clean[X_shuffled.index]


In [ ]:
tr_text_list = x_train_df['text'].values.tolist()
bow_preprocessor = CountVectorizer()
X = bow_preprocessor.fit_transform(tr_text_list)

5557


In [ ]:
# def custom_preprocessor(text):
#     text = re.sub(r"_", " ", text)  # Replace underscores with spaces
#     text = re.sub(r"\d+", "", text)  # Remove numbers
#     text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
#     return text.lower().strip()  # Convert to lowercase and trim space
 
# vectorizer = TfidfVectorizer(preprocessor=custom_preprocessor)
my_bow_classifier_pipeline = sklearn.pipeline.Pipeline([
    ('my_bow_feature_extractor', CountVectorizer(min_df=1, max_df=1.0, ngram_range=(1,1))),
    ('my_classifier', sklearn.linear_model.LogisticRegression(C=1.0, max_iter=20, random_state=random_state)),
])
my_parameter_grid_by_name = dict()
my_parameter_grid_by_name['my_bow_feature_extractor__min_df'] = [1, 2, 4]
my_parameter_grid_by_name['my_classifier__C'] = np.logspace(-4, 4, 9)
my_scoring_metric_name = 'accuracy'

In [92]:
# prng = np.random.RandomState(0)

# valid_ids = prng.choice(np.arange(N), size=round(N/5))

# valid_indicators_N = np.zeros(N)
# valid_indicators_N[valid_ids] = -1
# my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_N)
y_train_clean = list(y_train_clean)
grid_searcher = sklearn.model_selection.GridSearchCV(
    my_bow_classifier_pipeline,
    my_parameter_grid_by_name,
    scoring=my_scoring_metric_name,
    #cv=my_splitter,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state),
    refit=True)
grid_searcher.fit(tr_text_list, y_train_clean)

c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('my_bow_feature_extractor',
                                        CountVectorizer()),
                                       ('my_classifier',
                                        LogisticRegression(max_iter=20,
                                                           random_state=42))]),
             param_grid={'my_bow_feature_extractor__min_df': [1, 2, 4],
                         'my_classifier__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04])},
             scoring='accuracy')

In [96]:
gsearch_results_df = pd.DataFrame(grid_searcher.cv_results_).copy()

param_keys = ['param_my_bow_feature_extractor__min_df', 'param_my_classifier__C']

# Rearrange row order so it is easy to skim
gsearch_results_df.sort_values(param_keys, inplace=True)
gsearch_results_df[param_keys + ['mean_test_score', 'rank_test_score']]

,param_my_bow_feature_extractor__min_df,param_my_classifier__C,mean_test_score,rank_test_score
0,1,0.0001,0.604281,26
1,1,0.0010,0.668528,22
2,1,0.0100,0.700380,20
3,1,0.1000,0.716034,1
4,1,1.0000,0.715314,2
5,1,10.0000,0.712435,7
6,1,100.0000,0.713334,4
7,1,1000.0000,0.712975,5
8,1,10000.0000,0.712975,5
9,2,0.0001,0.604281,26


In [94]:
# Get the best-trained model (automatically refit on full training data)
best_model = grid_searcher.best_estimator_

# Best hyperparameters
best_params = grid_searcher.best_params_
print("Best Hyperparameters:", best_params)

# Best cross-validation score (average accuracy across folds)
best_score = grid_searcher.best_score_
print(f"Best CV Accuracy: {best_score:.4f}")

Best Hyperparameters: {'my_bow_feature_extractor__min_df': 1, 'my_classifier__C': 0.1}
Best CV Accuracy: 0.7160
